In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#print(loaded)
!pip install mne
!pip install mne[hdf5]
!pip install madmom
!pip install ffmpeg
!pip install mir_eval

#!conda activate mne


     |████████████████████████████████| 7.5 MB 4.7 MB/s 
     |████████████████████████████████| 20.0 MB 115 kB/s 
     |████████████████████████████████| 51 kB 5.8 MB/s 
  Created wheel for madmom: filename=madmom-0.16.1-cp37-cp37m-linux_x86_64.whl size=20935863 sha256=058aed37b8f8f32fd9057bcc34e1399f4e4109a54bc965f41fab083792a6a5fe
  Stored in directory: /root/.cache/pip/wheels/af/90/61/393ceef814b55b12d1b59b5ed3a2b2a3457a55d39b7363b975
Successfully built madmom
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6084 sha256=4a5bb226a47b372a8f90a80a930f29159973b7e34539835c5c6be1413ea7fda7
  Stored in directory: /root/.cache/pip/wheels/64/80/6e/caa3e16deb0267c3cbfd36862058a724144e19fdb9eb03af0f
Successfully built ffmpeg
     |████████████████████████████████| 90 kB 4.2 MB/s 
  Created wheel for mir-eval: filename=mir_eval-0.7-py3-none-any.whl size=100721 sha256=368d7b643f7ebf29d6eb4b7f21784801c495f5473bd9cf69b5b0bec73d14fa80
  Stored in directory: /root/.cache/pip/wh

In [ ]:
# from google.colab import drive
# drive.mount("/content/drive")
# %cd '/content/drive/MyDrive/2022 Spring/11-785 Introduction to Deep Learning/Project'

In [ ]:
from scipy import io
import os
import numpy as np
from sklearn.metrics import roc_auc_score
import mne
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import f1_score

# from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import torchaudio
# import wandb 
import madmom
import madmom.features.beats
import madmom.features.onsets
import mir_eval
import ffmpeg
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import gc
import zipfile
import pandas as pd
from tqdm import tqdm
import os
import datetime


In [ ]:
data_folder='/content/drive/MyDrive/11785 Project/data'
#data_folder = '/content/drive/MyDrive/11785-IDL/11785 Project/data'


#Restructuring the EEG folders

In [ ]:
timestamps = np.load(data_folder+'/ground_truths/timestamps.npy',allow_pickle=True)
len(timestamps)

10

In [ ]:
one_hot = np.load(data_folder+'/ground_truths/one_hot.npy',allow_pickle=True)
one_hot.shape

(10, 37500)

In [ ]:
data = io.loadmat(data_folder+'/users_mat/user0_restructured.mat')

In [ ]:
EEG = data['data']
EEG.shape

(10, 125, 37500)

In [ ]:
from scipy import io
loaded = io.loadmat(data_folder +'/users_mat/user0_restructured.mat',  struct_as_record=True)

# loaded = io.loadmat('/content/song21_Imputed.mat',  struct_as_record=True)
print(loaded.keys())
#dict_keys(['__header__', '__version__', '__globals__', 'mrk', 'cnt', 'nfo'])

dict_keys(['__header__', '__version__', '__globals__', 'data', 'user'])


In [ ]:
class UserDataset(torch.utils.data.Dataset):
     def __init__(self, data_path, partition= "train", user_id = "0"): # You can use partition to specify train or dev

        self.X_dir = data_path + "/users_mat/" # TODO: get mfcc directory path
        self.Y_dir = data_path + "/ground_truths/" # TODO: get transcript path

        self.X_files = os.listdir(self.X_dir)# TODO: list files in the mfcc directory
        print(self.X_files)
        if partition == 'train':
            self.X_files = [f for f in self.X_files if user_id not in f]
            val = 19
        else:
            self.X_files = [f for f in self.X_files if user_id in f]
            val = 1
        
        self.Y_onehot = np.load(data_path+'/ground_truths/one_hot.npy')
        self.Y_timestamps = np.load(data_path+'/ground_truths/timestamps.npy',allow_pickle=True)
        # self.Y_files = os.listdir(self.Y_dir)# TODO: list files in the transcript directory
        self.XXX = []
        self.YYY = []
  

        for x_file in self.X_files:
            Xtemp =  io.loadmat(data_folder+'/users_mat/'+x_file)
            data = Xtemp['data']
            data = mne.filter.filter_data(data,sfreq=100,l_freq=0.1,h_freq = 40)
            for i in data:
              #print("1",i.shape)
              temp = np.reshape(i,(300,125,125))
              #print("2",temp.shape)
              split_temp = np.split(temp,300)
              #print(len(split_temp))
              #print("3",split_temp[0].shape)
              self.XXX.extend(split_temp)
            
            

            for i in self.Y_onehot:
                temp = np.split(i, 300)
                #print(len(temp))
                self.YYY.extend(temp)
              
        

        print(len(self.YYY))
        print(len(self.XXX))


        pass

     def __len__(self):
        return len(self.X_files)

     def __getitem__(self, ind):

        return self.XXX[ind], self.YYY[ind]
    



In [ ]:
users = UserDataset(data_folder, partition= "train", user_id = "user0")
# print(users[0])

['user0_restructured.mat', 'user1_restructured.mat', 'user2_restructured.mat', 'user3_restructured.mat', 'user4_restructured.mat', 'user5_restructured.mat', 'user6_restructured.mat', 'user7_restructured.mat', 'user8_restructured.mat', 'user9_restructured.mat', 'user10_restructured.mat', 'user11_restructured.mat', 'user12_restructured.mat', 'user13_restructured.mat', 'user14_restructured.mat', 'user15_restructured.mat', 'user16_restructured.mat', 'user17_restructured.mat', 'user18_restructured.mat', 'user19_restructured.mat']
Setting up band-pass filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency:

KeyboardInterrupt: ignored

In [ ]:
class UserRNNValidDataset(torch.utils.data.Dataset):
    def __init__(self, data_path, user_id = "0"): # You can use partition to specify train or dev
        self.X_dir = data_path + "/users_mat/" # TODO: get mfcc directory path
        self.Y_dir = data_path + "/ground_truths/" # TODO: get transcript path

        self.X_files = os.listdir(self.X_dir)# TODO: list files in the mfcc directory
        self.X_files = [f for f in self.X_files if user_id in f]
        val = 1
        print(self.X_files)
        
        self.Y_onehot = np.load(data_path+'/ground_truths/one_hot.npy')
        self.Y_timestamps = np.load(data_path+'/ground_truths/timestamps.npy',allow_pickle=True)
        # self.Y_files = os.listdir(self.Y_dir)# TODO: list files in the transcript directory
        self.XXX = []
        self.YYY = []
        
        for x_file in tqdm(self.X_files):
            Xtemp =  io.loadmat(data_folder+'/users_mat/'+x_file)
            data = Xtemp['data']
            data = mne.filter.filter_data(data,sfreq=100,l_freq=0.1,h_freq = 40)
            for i in data:
                x = i.T
                # temp = np.reshape(i,(300,125,125))
                # split_temp = np.split(temp,300)
                self.XXX.append(x)


            for i in self.Y_onehot:
                # y = np.reshape(i,(300,125))
                # temp = np.split(i, 300)
                self.YYY.append(i)

                # self.YYY.extend(temp)

        assert(len(self.YYY) == len(self.XXX))

    def __len__(self):
        return len(self.XXX)

    def __getitem__(self, ind):
        return self.XXX[ind], self.YYY[ind]
    
    def collate_fn(batch):
        batch_x = [torch.from_numpy(x) for x,y in batch]
        batch_y = [torch.from_numpy(y) for x,y in batch]

        return torch.stack(batch_x,dim=0).squeeze(dim=0),torch.stack(batch_y,dim=0).squeeze(dim=0)



In [ ]:
class UserRNNDataset(torch.utils.data.Dataset):
    def __init__(self, data_path, partition= "train", user_id = "0",context=0): # You can use partition to specify train or dev
        self.X_dir = data_path + "/users_mat/" # TODO: get mfcc directory path
        self.Y_dir = data_path + "/ground_truths/" # TODO: get transcript path
        self.context = context
        self.X_files = os.listdir(self.X_dir)# TODO: list files in the mfcc directory
        print(user_id)
        if partition == 'train':
            self.X_files = [f for f in self.X_files if user_id not in f]
            val = 19
        else:
            self.X_files = [f for f in self.X_files if user_id in f]
            val = 1
        print(self.X_files)
        
        self.Y_onehot = np.load(data_path+'/ground_truths/one_hot.npy')
        self.Y_timestamps = np.load(data_path+'/ground_truths/timestamps.npy',allow_pickle=True)
        # self.Y_files = os.listdir(self.Y_dir)# TODO: list files in the transcript directory
        XXX = []
        self.YYY = []
        
        for x_file in self.X_files:
            Xtemp =  io.loadmat(data_folder+'/users_mat/'+x_file)
            data = Xtemp['data']
            data = mne.filter.filter_data(data,sfreq=100,l_freq=0.1,h_freq = 40)
            for i in data:
                x = i.T
                temp = np.reshape(i,(300,125,125))
                split_temp = np.split(temp,300)
                XXX.extend(split_temp)
            
            del Xtemp,data

            for i in self.Y_onehot:
                y = np.reshape(i,(300,125))
                temp = np.split(i, 300)
                self.YYY.extend(temp)

        self.XX =np.array(XXX)
        # print(self.XX.shape)
        if context!=0:
            self.add_blocks = context//125 + 1
            self.XX = np.pad(self.XX,((self.add_blocks,self.add_blocks),(0,0),(0,0),(0,0)))
            # print("post pad",self.XX.shape)
        
        # assert(len(self.YYY) == len(self.XXX))

    def __len__(self):
        return len(self.YYY)

    def __getitem__(self, ind):
        if self.context!=0:
            extracted = self.XX[ind:ind+(self.add_blocks*2)+1,0]
            pre_x = extracted[0,-self.context%125:,:]
            post_x = extracted[-1,:self.context%125,:]
            # print("pre",pre_x.shape)
            # print("post",post_x.shape)
            middle_x = extracted[1:-1,:,:].reshape((-1,125))
            # print("middle",middle_x.shape)
            final_x = np.vstack((pre_x,middle_x,post_x))
            # print('final_x',final_x.shape)
            return final_x,self.YYY[ind]
        else:
            return self.XXX[ind], self.YYY[ind]
    
    def collate_fn(batch):
        batch_x = [torch.from_numpy(x) for x,y in batch]
        batch_y = [torch.from_numpy(y) for x,y in batch]

        return torch.stack(batch_x,dim=0).squeeze(),torch.stack(batch_y,dim=0).squeeze()



In [ ]:
user_id=0
context = 5
# train_dataset = UserRNNDataset(data_folder,'train','user'+str(user_id),context = context)
# val_dataset = UserRNNDataset(data_folder,partition="valid",user_id='user'+str(user_id),context = context)
# train_loader =torch.utils.data.DataLoader(train_dataset,batch_size=32,collate_fn = UserRNNDataset.collate_fn,shuffle=True,num_workers=4)
# val_loader = torch.utils.data.DataLoader(val_dataset,batch_size=32,collate_fn = UserRNNDataset.collate_fn,shuffle=False,num_workers=4)


In [ ]:
# class Baseline(torch.nn.Module):
#     def __init__(self,context=0):
#         super(Baseline, self).__init__()
#         # self.flatten = nn.Flatten()
#         self.context_embedding = nn.Conv1d(in_channels=125, out_channels=125,kernel_size = 2*context+1,padding="valid",stride=1,groups = 125)
#         self.rnn = nn.GRU(input_size = 125,hidden_size=64,num_layers=2,batch_first=True)

#         self.layers = nn.Sequential(
#             nn.Linear(64, 1), # try with 125 output
#         )

#     def forward(self, x):
#         # x = torch.flatten(x,start_dim=-2)
#         # print('forwaard')
#         trans_x = torch.permute(x,(0,2,1))
#         # print("perm inp",trans_x.shape)
#         x_out = self.context_embedding(trans_x)
#         # print("out of conv",x_out.shape)
#         x = torch.permute(x_out,(0,2,1))
#         # print("conv op permute",x.shape)
#         out,h_n =self.rnn(x)
#         x = self.layers(out)
#         # print("")
#         return x


In [ ]:
# class Baseline(torch.nn.Module):
#     def __init__(self,context=0):
#         super(Baseline, self).__init__()
#         # self.flatten = nn.Flatten()
#         self.context_embedding = nn.Sequential(
#             nn.Conv1d(in_channels=125, out_channels=125,kernel_size = 2*context+1,padding="valid",stride=1,groups = 125),
#             nn.BatchNorm1d(125),
#             nn.LeakyReLU(),)
#         self.rnn = nn.GRU(input_size = 125,hidden_size=64,num_layers=2,batch_first=True)

#         self.layers = nn.Sequential(
#             nn.Linear(64, 64),
#             nn.BatchNorm1d(64),
#             nn.LeakyReLU(),
#             nn.Dropout(0.35),
#             nn.Linear(64, 1), # try with 125 output
#         )

#     def forward(self, x):
#         trans_x = torch.permute(x,(0,2,1))
#         x_out = self.context_embedding(trans_x)
#         x = torch.permute(x_out,(0,2,1))
#         out,h_n =self.rnn(x)
#         x = self.layers(out)
#         # print("")
#         return x


In [ ]:
class Baseline(torch.nn.Module):
    def __init__(self,context=0):
        super(Baseline, self).__init__()
        # self.flatten = nn.Flatten()
        #self.context_embedding = nn.Conv1d(in_channels=125, out_channels=125,kernel_size = 2*context+1,padding="valid",stride=1,groups = 125)
        
        self.context_embedding = nn.Sequential(
            nn.Conv1d(in_channels=125,out_channels=125,kernel_size=2*context+1,stride=1, padding="valid",groups = 125),
        )
        
        #self.rnn = nn.GRU(input_size = 125,hidden_size=64,num_layers=2,batch_first=True)
        self.lstm = nn.LSTM(125,125, 3, batch_first = True, dropout = 0.4, bidirectional=True)

        #self.layer_norm = nn.LayerNorm(125)

        self.layers = nn.Sequential(
            nn.Linear(250, 2056),
            nn.LeakyReLU(),
            nn.Dropout(0.3),
            nn.Linear(2056, 1), # try with 125 output
        )

    def forward(self, x):
        trans_x = torch.permute(x,(0,2,1))
        x_out = self.context_embedding(trans_x)
        x = torch.permute(x_out,(0,2,1))
        out,h_n =self.lstm(x)
        #out = self.layer_norm(out)
        x = self.layers(out)
        # print("")
        return x


In [ ]:
# del model
# model = Baseline(context=context).cuda()


In [ ]:
# for data in val_loader:
#     x=data[0].cuda()
#     y=data[1].cuda()
    
#     # x=data[0].cuda().unsqueeze(dim=0)
#     # y=data[1].cuda().unsqueeze(dim=0)
#     print('x',x.shape)
#     print('y',y.shape)
#     out = model(x.float())

#     print('out',out.shape)
#     print(y.flatten().shape)

#     print(out.flatten().shape)
#     # print(min(out),max(out))
#     out2 =nn.Sigmoid()(out)
#     print(roc_auc_score(y.cpu().detach().numpy().flatten(),out2.cpu().detach().numpy().flatten()))
#     break

In [ ]:
# out

In [ ]:
# model(x.float()).shape

In [ ]:
def validate(model,val_loader,criterion):
    model.eval()
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, leave=False, position=0, desc='Validation')

    f1=[]

    recall=[]
    precision=[]
    total_val_loss = 0
    ground_truth=np.empty((1,0))
    predictions = np.empty((1,0))
    # print("Validation")
    total_raw_score=0
    for idx,(x,y) in tqdm(enumerate(val_loader)):
    # for idx,(x,y) in enumerate(val_loader):

        ground_truth = np.append(ground_truth,y.reshape(1,-1))
        x = x.float().cuda()
        y = y.float().cuda()

        with torch.no_grad():
            pd_one_hot = model(x)
            predictions = np.append(predictions,nn.Sigmoid()(pd_one_hot).squeeze().cpu().detach().numpy().reshape(1,-1))
            val_loss = criterion(pd_one_hot.squeeze(),y)
            y_np = y.cpu().numpy()
            pred_np = nn.Sigmoid()(pd_one_hot).squeeze().cpu().detach().numpy()
            total_raw_score += roc_auc_score(y_np.flatten(),pred_np.flatten())
            # pd_one_hot = nn.Sigmoid()(pd_one_hot)
            # pd_one_hot = torch.flatten(pd_one_hot)
            # pd_one_hot = pd_one_hot.cpu().numpy()
        total_val_loss +=val_loss
        # out = madmom.features.onsets.peak_picking(pd_one_hot,0.5)
        # predicted_ts = out/125
        # if len(predicted_ts)==0:
        #     F,P,R =0.0,0.0,0.0
        # else:
        #     F,P,R = mir_eval.onset.f_measure(gt_ts,predicted_ts,window=1)
        # f1.append(F)
        # precision.append(P)
        # recall.append(R)
        batch_bar.set_postfix(roc_score="{:.04f}".format(float(total_raw_score / (idx + 1))),loss="{:.04f}".format(float(total_val_loss / (idx + 1))), lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
        batch_bar.update()
        del x,y,pd_one_hot,val_loss
        torch.cuda.empty_cache()
    # print(predictions.shape)
    batch_bar.close()

    for idx in range(10):
        print("")
        gt_ts = val_loader.dataset.Y_timestamps[idx]
        song_pred = predictions[idx*37500:(idx+1)*37500]

        # print(song_pred.shape,gt_ts.shape)
        #print(len(gt_ts))
        out = madmom.features.onsets.peak_picking(song_pred,0.055)
        predicted_ts = out/125
        #print(len(predicted_ts))
        if len(predicted_ts)==0:
            F,P,R =0.0,0.0,0.0
        else:
            F,P,R = mir_eval.onset.f_measure(gt_ts,predicted_ts,window=0.1)
        f1.append(F)
        precision.append(P)
        recall.append(R)

    # print(f1)
    # print(precision)
    # print(recall)
    model.train()
    return f1,precision,recall,total_val_loss/len(val_loader)


In [ ]:
epoch = 6
scores = {"f":[],'p':[],'r':[]}

In [ ]:
del train_dataset,val_dataset,train_loader,val_loader,model,criterion
del optimizer

In [ ]:
def train(model,epoch,train_loader,val_loader,criterion,optimizer, scheduler):
    for e in range(epoch):
        print("Epoch ",e)
        batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Training')
        total_loss=0
        for idx,data in enumerate(train_loader):
            model.train()
            optimizer.zero_grad()
            x=data[0].float().cuda()
            y=data[1].float().cuda()
            out = model(x)
            out=out.squeeze()
            loss = criterion(out,y)
            loss.backward()
            optimizer.step()
            total_loss+=loss

            batch_bar.set_postfix(loss="{:.04f}".format(float(total_loss / (idx + 1))), lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
            batch_bar.update()

            del data,x,y,out,loss
            torch.cuda.empty_cache()
        batch_bar.close()

        f,p,r,val_loss  = validate(model,val_loader,criterion)
        

        print("Epoch {}/{}: Train Loss {:.04f}, Learning Rate {:.04f}, Validation Loss {:.04f}, F1 Score {:.04f}, Precision {:.04f}, Recall {:.04f}".format(
            e,
            epoch,
            float(total_loss / (len(train_loader))),
            float(optimizer.param_groups[0]['lr']),val_loss,np.mean(f),np.mean(p),np.mean(r)))
        
        scheduler.step(val_loss)
    return np.mean(f),np.mean(p),np.mean(r)

In [ ]:
model = Baseline(context=context).cuda()
#f,p,r = train(model,epoch,train_loader,val_loader,criterion,optimizer, scheduler)


In [ ]:
import gc
valid_ids = [0,5,10,15]
# for user_id in valid_ids:
for user_id in valid_ids:
    print('user_id', user_id)
    gc.collect()
    context = 60
    train_dataset = UserRNNDataset(data_folder,'train','user'+str(user_id),context = context)
    val_dataset = UserRNNDataset(data_folder,partition="valid",user_id='user'+str(user_id),context = context)
    train_loader =torch.utils.data.DataLoader(train_dataset,batch_size=32,collate_fn = UserRNNDataset.collate_fn,shuffle=True,num_workers=4)
    val_loader = torch.utils.data.DataLoader(val_dataset,batch_size=32,collate_fn = UserRNNDataset.collate_fn,shuffle=False,num_workers=4)
    model = Baseline(context=context).cuda()
    #model = Network(context=context).cuda()
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(),lr = 1e-3)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, threshold = 0.1, factor = 0.3, patience = 2, mode = 'min')


    f,p,r = train(model,epoch,train_loader,val_loader,criterion,optimizer, scheduler)
    scores['f'].append(f)
    scores['p'].append(p)
    scores['r'].append(r)

    del train_dataset,val_dataset,train_loader,val_loader,model,criterion,optimizer
    torch.cuda.empty_cache()
    gc.collect()


user_id 0


user0
['user1_restructured.mat', 'user2_restructured.mat', 'user3_restructured.mat', 'user4_restructured.mat', 'user5_restructured.mat', 'user6_restructured.mat', 'user7_restructured.mat', 'user8_restructured.mat', 'user9_restructured.mat', 'user10_restructured.mat', 'user11_restructured.mat', 'user12_restructured.mat', 'user13_restructured.mat', 'user14_restructured.mat', 'user15_restructured.mat', 'user16_restructured.mat', 'user17_restructured.mat', 'user18_restructured.mat', 'user19_restructured.mat']
Setting up band-pass filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter 

Validation:  96%|█████████▌| 90/94 [00:03<00:00, 33.18it/s, loss=0.1755, lr=0.0010, roc_score=0.5268]
94it [00:03, 29.50it/s]












Epoch 0/6: Train Loss 0.1782, Learning Rate 0.0010, Validation Loss 0.1765, F1 Score 0.6112, Precision 0.6398, Recall 0.6039
Epoch  1


Validation:  96%|█████████▌| 90/94 [00:03<00:00, 32.80it/s, loss=0.1753, lr=0.0010, roc_score=0.5310]
94it [00:03, 29.49it/s]


...











Epoch 0/6: Train Loss 0.1783, Learning Rate 0.0010, Validation Loss 0.1769, F1 Score 0.6465, Precision 0.5998, Recall 0.7247
Epoch  1


Validation:  99%|█████████▉| 93/94 [00:04<00:00, 30.92it/s, loss=0.1765, lr=0.0010, roc_score=0.5335]
94it [00:03, 26.33it/s]












Epoch 1/6: Train Loss 0.1729, Learning Rate 0.0010, Validation Loss 0.1765, F1 Score 0.5970, Precision 0.6280, Recall 0.5870
Epoch  2


Training:  12%|█▏        | 207/1782 [00:14<01:37, 16.09it/s, loss=0.1699, lr=0.0010]

KeyboardInterrupt: ignored

In [ ]:
print("Final F1 score : ", sum(scores['f'][:])/4)
print("Final precision score : ",sum(scores['p'][:])/4)
print("Final recall score : ",sum(scores['r'][:])/4)
print(scores)

In [ ]:
scores2 = scores.copy()

In [ ]:
out = validate(model,val_loader,criterion)

In [ ]:
val_dataset.Y_timestamps[0],val_dataset.Y_timestamps[1]

In [ ]:
val_dataset[0][1][1]

In [ ]:
val_dataset[1][1][0]


In [ ]:
a=np.array([0,3.7,4,4.1,0.1,2.3,2,0,0])

In [ ]:
out = madmom.features.onsets.peak_picking(a,0.5)

In [ ]:
out

In [ ]:
a=np.array([0,234,5685,23424])/125

In [ ]:
a

In [ ]:
b =val_loader.dataset.Y_timestamps[0]

In [ ]:
mir_eval.onset.f_measure(b,
                                   a)

In [ ]:
dataset = UserRNNDataset(data_folder,partition='val',user_id="user0")

In [ ]:
len(dataset)

In [ ]:
x,y = dataset[0]

In [ ]:
x.shape

In [ ]:
y.shape

In [ ]:
#dict_keys(['__header__', '__version__', '__globals__', 'data21', 'fs', 'subs21'])

dat = np.transpose(np.array(loaded['data21']), (2,0,1)) #Putting into 20 x 125 x N

print(dat.shape)
print(loaded['data21'].shape) #Actual Data
#(20 subjects, 125 channels and N time steps)
print(loaded['fs'].shape)
print(loaded['fs']) #fs is just for number of channels
print(loaded['subs21'].shape) #Subjects


#sfreq = loaded['data21'][125]
EEGdata   = dat
nsubjects, nchannels, ntimesteps = EEGdata.shape


In [ ]:
pad_length = 37500 - ntimesteps

EEGdata = np.pad(EEGdata, ((0,0) , (0,0), (0, pad_length)), constant_values=0)

print("padded EEG data shape: ", EEGdata.shape)

In [ ]:
#Segment into one second blocks
split_EEGdata = np.dsplit(EEGdata, 300)
print(split_EEGdata[0].shape)

In [ ]:
#I think the split worked correctly?
print(len(split_EEGdata[0][0][0]))
print(len(split_EEGdata))

In [ ]:
#we filter the EEG data with a bandpass filter (0.1–
#40 Hz) and zero-pad the dataset uniformly to a length of 37500
#samples (5 min), since the longest song in the dataset is around
#five minutes long

!pip install eeg-filters



In [ ]:

from scipy.signal import butter, lfilter

def butter_bandpass(lowcut, highcut, fs, order=5):
    return butter(order, [lowcut, highcut], fs=fs, btype='band')

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y


import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import freqz

# Sample rate and desired cutoff frequencies (in Hz).
fs = 5000.0
lowcut = 500.0
highcut = 1250.0

# Plot the frequency response for a few different orders.
plt.figure(1)
plt.clf()
for order in [3, 6, 9]:
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    w, h = freqz(b, a, fs=fs, worN=2000)
    plt.plot(w, abs(h), label="order = %d" % order)

plt.plot([0, 0.5 * fs], [np.sqrt(0.5), np.sqrt(0.5)],
          '--', label='sqrt(0.5)')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Gain')
plt.grid(True)
plt.legend(loc='best')

# Filter a noisy signal.
T = 0.05
nsamples = T * fs
t = np.arange(0, nsamples) / fs
a = 0.02
f0 = 600.0
x = 0.1 * np.sin(2 * np.pi * 1.2 * np.sqrt(t))
x += 0.01 * np.cos(2 * np.pi * 312 * t + 0.1)
x += a * np.cos(2 * np.pi * f0 * t + .11)
x += 0.03 * np.cos(2 * np.pi * 2000 * t)

#x = EEGdata[0]

plt.figure(2)
plt.clf()
plt.plot(t, x, label='Noisy signal')

y = butter_bandpass_filter(x, lowcut, highcut, fs, order=6)
plt.plot(t, y, label='Filtered signal (%g Hz)' % f0)
plt.xlabel('time (seconds)')
plt.hlines([-a, a], 0, T, linestyles='--')
plt.grid(True)
plt.axis('tight')
plt.legend(loc='upper left')

plt.show()


Attempting to Annotate Music with MADMOM ODF RNN

In [ ]:
import madmom
import madmom.features.beats
import madmom.features.onsets
import ffmpeg

In [ ]:
#from scipy.io import wavfile
#samplerate, data = wavfile.read('/content/first_fires.wav')

In [ ]:
#proc1.reset()
proc1 = madmom.features.onsets.OnsetPeakPickingProcessor(fps = 100) #What should fps be?
act1 = madmom.features.onsets.RNNOnsetProcessor()(data_folder + '/content/first_fires.wav') 
onset_values = proc1(act1) 
print(onset_values)
gt = np.zeros(37500)
idx_list=[]
for onset in (onset_values):
    idx_list.append(onset*125)
print(idx_list)
idx_list = np.array(idx_list)

idx_list = np.round(idx_list,0).astype('int64')
print(idx_list)

gt[idx_list] = 1

In [ ]:
print(len(onset_values))

import os
import wave

with wave.open(data_folder + '/songs_wav/first_fires.wav', "rb") as wave_file:
  frame_rate = wave_file.getframerate()
print(frame_rate)

In [ ]:
#lmao this is so wrong

song_labels = np.zeros(37500)

for i in onset_values:
  song_labels[int(i)] = 1

In [ ]:
print(song_labels[-1000:])

In [ ]:
split_song_labels = np.split(song_labels, 300)
print(split_song_labels[0].shape)

In [ ]:
print(len(split_song_labels))

Fully Connected Network

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
from torch.nn.modules.batchnorm import BatchNorm2d
class Network(torch.nn.Module): #All your models should inherit from torch.nn.Module
    def __init__(self, context = 0):
        super(Network, self).__init__()
        # TODO: Please try different architectures
        
        layers = [
            nn.Linear(15625, 256),
            nn.ReLU(),
            nn.Linear(256, 125)]

        self.laysers = nn.Sequential(*layers)


    def forward(self, A0):
        x = self.laysers(A0)
        return x

In [ ]:
#train_loader = torch.utils.data.DataLoader(split_EEGdata, batch_size=50, shuffle=False)
out = Network(split_EEGdata[0].flatten())

In [ ]:
class ToyDataset(torch.utils.data.Dataset):
    def __init__(self):
        self.XXX = np.arange(200).reshape(40,-1)
        self.YYY = self.XXX * 2

    def __len__(self):
        return len(self.XXX)

    def __getitem__(self, ind):
        return self.XXX[ind], self.YYY[ind]
    
    def collate_fn(batch):
        batch_x = [torch.from_numpy(x) for x,y in batch]
        batch_y = [torch.from_numpy(y) for x,y in batch]

        return torch.stack(batch_x,dim=0).squeeze(),torch.stack(batch_y,dim=0).squeeze()
    

In [ ]:
data = ToyDataset()

In [ ]:
data[0]

In [ ]:
loader = torch.torch.utils.data.DataLoader(data,batch_size=32,collate_fn = ToyDataset.collate_fn,shuffle=False,num_workers=4)

In [ ]:
final_y = np.empty((1,0))
for x,y in loader:
    print(x.reshape(1,-1))
    final_y = np.append(final_y,y.reshape(-1,1))

In [ ]:
final_y

In [ ]:
from sklearn.metrics import f1_score


In [ ]:
y_pred = np.zeros((3,10))
y_true = np.ones((3,10))

In [ ]:
y_pred[1][4] = 1
y_pred[2][5] = 1
y_pred[0][4] = 1
y_pred[2][0] = 1
y_pred[0][0] = 1
y_pred[1][0] = 1
y_pred[0][4] = 1
y_pred[1][8] = 1


In [ ]:
y_true[1][8]=0
y_true[0][1]=0
y_true[0][1]=0
y_true[0][8]=0
y_true[2][4]=0
y_true[0][2]=0
y_true[1][7]=0


In [ ]:
f1_score(y_pred.flatten(),y_true.flatten())

In [ ]:
a = np.arange(20).reshape(4,5)
b = np.arange(100,120).reshape(4,5)

In [ ]:
x=[]
x.append(a)
x.append(b)

In [ ]:
m = np.array(x)

In [ ]:
np.pad(m,((2,2),(0,0),(0,0)))

In [ ]:
a =np.arange(2*3*5).reshape(2,3,5)

In [ ]:
a

In [ ]:
a = np.arange(8).reshape((2,4))
b = np.arange(12).reshape(3,4)
c = np.arange(8).reshape((2,4))


In [ ]:
b

In [ ]:
np.vstack((a,b,c)).shape